In [9]:
#------------------- Import packages -------------------#
import os
from os import listdir
from os.path import isfile, join

# Data wrangling
import pandas as pd
import numpy as np

# Vizualizations
import matplotlib.pyplot as plt
import matplotlib.dates as matdates

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as gobjects
from plotly.subplots import make_subplots

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline()


# Working with datetimes
import datetime
import time

# Utitlities
import yfinance as yf
import warnings
warnings.simplefilter('ignore')

# Constants

In [2]:
_DATAPATH = 'data/'

startDate_str = '2017-01-01'
endDate_str = '2022-01-01'
SDATE = pd.to_datetime(startDate_str)
EDATE = pd.to_datetime(endDate_str)


# Helper functions | Download Data

In [3]:
# Get column from file
def get_column(file, name):
    try:
        if 'csv' in file:
            df = pd.read_csv(file)
        else:
            df = pd.read_excel(file)
    except FileNotFoundError:
        print('There is no such file: {}'.format(file))
    else:
        return df[name]

In [4]:
tickers = get_column('SP500.csv', 'Symbol')
print('Number of tickers extracted: {}'.format(len(tickers)))

Number of tickers extracted: 505


In [7]:
def download_data_yahoo(tickers, period):
    def collect_data(folder, ticker, period):
        asset = yf.Ticker(ticker)

        try:
            # print('Collecting data for: {}'.format(ticker))
            df = asset.history(period = period)
            if df.shape[0] == 0:
                return 0
            else:
                savedFile = folder + ticker.replace('.', '_') + '.csv'
                df.to_csv(savedFile)
                return 1

        except:
            print('Unsuccessful: {}'.format(ticker))
            return 0

    count = 0
    for ticker in tickers:
        count += collect_data(_DATAPATH, ticker, period)

    print('Downloaded data for {} stocks'.format(count))


In [8]:
download_data_yahoo(tickers, '5y')

- ALXN: No data found, symbol may be delisted
- AGN: No data found, symbol may be delisted
- APC: No data found, symbol may be delisted
- BHGE: No data found, symbol may be delisted
- BBT: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- COG: No data found, symbol may be delisted
- CBG: No data found for this date range, symbol may be delisted
- CBS: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- CTL: No data found, symbol may be delisted
- XEC: No data found, symbol may be delisted
- CXO: No data found, symbol may be delisted
- DWDP: No data found, symbol may be delisted
- DPS: No data found for this date range, symbol may be delisted
- ETFC: No data found, symbol may be delisted
- FLIR: No data found, symbol may be delisted
- GGP: No data found for this date range, symbol may be delisted
- HRS: No data found, symbol may be delisted
- JEC: No d

# Helper Functions | Daily returns and Cummulative return

In [10]:
def calculate_daily_return(df):
    df['return_daily'] = (df['Close'] / df['Close'].shift(1)) - 1
    #df.to_csv(_DATAPATH + ticker + '.csv')
    return df

In [11]:
def calculate_cummulative_return(df):
    df['return_cummulative'] = (1 + df['return_daily']).cumprod()
    return df

# Helper Functions | Bollinger Bands

Bollinger Bands plot 2 lines using a moving average and the standard deviation defines how far apart the lines are. They also are used to define if prices are to high or low. When bands tighten it is believed a sharp price move in some direction. Prices tend to bounce off of the bands which provides potential market actions.

A strong trend should be noted if the price moves outside the band. If prices go over the resistance line it is in overbought territory and if it breaks through support it is a sign of an oversold position.

You normally use 20 sessions when using them.

 - High Band (Resistance) = Simple Moving Average + 2 * Standard Deviation
 - Low Band (Support) = Simple Moving Average - 2 * Standard Deviation

In [12]:
def calculate_bollinger_bands(df):
    df['band_baseline'] = df['Close'].rolling(window = 20).mean()
    df['band_upper'] = df['band_baseline'] + 1.96 * df['Close'].rolling(window = 20).std()
    df['band_lower'] = df['band_baseline'] - 1.96 * df['Close'].rolling(window = 20).std()
    return df

# Helper Functions | Ichimoku
The Ichimoku (One Look) is considered an all in one indicator. It provides information on momentum, support and resistance. It is made up of 5 lines. If you are a short term trader you create 1 minute or 6 hour. Long term traders focus on day or weekly data.

- Conversion Line (Tenkan-sen) : Represents support, resistance and reversals. Used to measure short term trends.
- Baseline (Kijun-sen) : Represents support, resistance and confirms trend changes. Allows you to evaluate the strength of medium term trends. Called the baseline because it lags the price.
- Leading Span A (Senkou A) : Used to identify future areas of support and resistance
- Leading Span B (Senkou B) : Other line used to identify suture support and resistance
- Lagging Span (Chikou) : Shows possible support and resistance. It is used to confirm signals obtained from other lines.
- Cloud (Kumo) : Space between Span A and B. Represents the divergence in price evolution.

Formulas
- Lagging Span = Price shifted back 26 periods
- Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
- Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
- Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
- Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)

In [13]:
def calculate_ichimoku(df):
    # Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
    valueH_9 = df['High'].rolling(window = 9).max()
    valueL_9 = df['Low'].rolling(window = 9).max()
    df['Conversion'] = (valueH_9 + valueL_9)/2

    # Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
    valueH_26 = df['High'].rolling(window = 26).max()
    valueL_26 = df['Low'].rolling(window = 26).max()
    df['Baseline'] = (valueH_26 + valueL_26)/2

    # Span A = (Conversion Value + Base Value)/2 (26 Sessions)
    df['Span A'] = ((df['Conversion'] + df['Baseline']) / 2)

    # Span B = (Conversion Value + Base Value)/2 (52 Sessions)
    valueH_52 = df['High'].rolling(window = 52).max()
    valueL_52 = df['Low'].rolling(window = 52).max()
    df['Span B'] = ((valueH_52 + valueL_52) / 2).shift(26)

    # Lagging Span = Price shifted back 26 periods
    df['Lagging Span'] = df['Close'].shift(-26)

    return df

# Helper Functions | Create Dataframe

In [14]:
# Create dataframe from CSV
def create_df(ticker):
    try: 
        df = pd.read_csv(_DATAPATH + ticker + '.csv')
    except FileNotFoundError:
        print('CSV for {} does not exist'.format(ticker))
    else:
        return df

# Drive Code

In [15]:
# Stocks downloaded save to to list
tickers = [file.split('.')[0] for file in listdir(_DATAPATH) if isfile(join(_DATAPATH, file))]
# tickers.remove('.ds_Store')

tickers.sort()
print('Number of stock files: {}'.format(len(tickers)))

Number of stock files: 466


In [16]:
for ticker in tickers:
    try:
        df_new = create_df(ticker)
        df_new = calculate_daily_return(df_new)
        df_new = calculate_cummulative_return(df_new)
        df_new = calculate_bollinger_bands(df_new)
        df_new = calculate_ichimoku(df_new)
        df_new.to_csv(_DATAPATH + ticker + '.csv')
    except Exception as ex:
        print(ex)

# Data Vizualisation

### Plot Bollinger Bands

In [46]:
def plot_boll_bands(df, ticker):
    fig = gobjects.Figure()

    candle = gobjects.Candlestick(x = df.index, open = df['Open'], high = df['High'], low = df['Low'], close = df['Close'], name = 'Candlestick')

    middlerBand = gobjects.Scatter(x = df.index, y = df['band_baseline'], line = dict(color = 'rgba(0, 0, 250, 0.75)', width = 0.8), name = 'Middle Band')
    upperBand = gobjects.Scatter(x = df.index, y = df['band_upper'], line = dict(color = 'rgba(250, 0, 0, 0.75)', width = 1), name = 'Upper Band')
    lowerBand = gobjects.Scatter(x = df.index, y = df['band_lower'], line = dict(color = 'rgba(0, 250, 0, 0.75)', width = 1), name = 'Lower Band')

    fig.add_trace(candle)
    fig.add_trace(middlerBand)
    fig.add_trace(upperBand)
    fig.add_trace(lowerBand)

    fig.update_xaxes(title = 'Date', rangeslider_visible = True)
    fig.update_yaxes(title = 'Price')

    fig.update_layout(title = ticker + ' Bollinger Bands', height = 600, width = 1200, showlegend = True)
    fig.show()

In [75]:
test_df = create_df('AMD')
plot_boll_bands(test_df, 'AMD')

# Plot Ichimoku

In [49]:
def plot_ichimoku(df):
    # As spans are going to cross eac hother, we need to change the ccolors accordingly
    def get_color_on_cross(flag):
        if flag >= 1:
            return 'rgba(0, 250, 0, 0.4)'

        return 'rgba(250, 0, 0, 0.4)'
    
    candle = gobjects.Candlestick(x = df.index, open = df['Open'], high = df['High'], low = df['Low'], close = df['Close'], name = 'Candlestick')

    df_new = df.copy()
    fig = gobjects.Figure()
    df['Flag'] = np.where(df['Span A'] > df['Span B'], 1, 0)
    df['Group'] = df['Flag'].ne(df['Flag'].shift()).cumsum()
    df = df.groupby('Group')

    newList = []
    for name, data in df:
        newList.append(data)

    for df in newList:
        fig.add_traces(gobjects.Scatter(x = df.index, y = df['Span A'], line = dict( color = 'rgba(0,0,0,0)')))
        fig.add_traces(gobjects.Scatter(x = df.index, y = df['Span B'], line = dict( color = 'rgba(0,0,0,0)'), fill = 'tonexty', fillcolor = get_color_on_cross(df['Flag'].iloc[0])))

    baseline = gobjects.Scatter(x = df_new.index, y = df_new['Baseline'], line = dict(color = 'pink', width = 2), name = 'Baseline')
    conversion = gobjects.Scatter(x = df_new.index, y = df_new['Conversion'], line = dict(color = 'black', width = 1), name = 'Conversion')
    lagging = gobjects.Scatter(x = df_new.index, y = df_new['Lagging Span'], line = dict(color = 'purple', width = 2), name = 'Lagging Span')
    spanA = gobjects.Scatter(x = df_new.index, y = df_new['Span A'], line = dict(color = 'green', width = 2, dash = 'dot'), name = 'Span A')
    spanB = gobjects.Scatter(x = df_new.index, y = df_new['Span B'], line = dict(color = 'red', width = 2, dash = 'dot'), name = 'Span B')

    fig.add_trace(candle)
    fig.add_trace(baseline)
    fig.add_trace(conversion)
    fig.add_trace(lagging)
    fig.add_trace(spanA)
    fig.add_trace(spanB)
    fig.update_layout(height = 600, width = 1200, showlegend = True)
    fig.show()

In [76]:
test_df = create_df('AAPL')
plot_ichimoku(test_df)

In [17]:
from bs4 import BeautifulSoup
import requests
# retrieve the web page and parse the contents
mainpage = requests.get('https://coinmarketcap.com/')
soup = BeautifulSoup(mainpage.content, 'html.parser')
whatis = soup.find_all("p", {"class" : "sc-1eb5slv-0 gGIpIK coin-item-symbol"})
# extract elements from the contents


In [18]:
def extract_crypto_ticker(tag):
    try:
        tag = str(tag)
        return tag.split('>')[1].split('<')[0]
    except TypeError:
        print('Pass')

In [19]:
crypto_tickers = [extract_crypto_ticker(x) for x in whatis  if extract_crypto_ticker(x) != '']

# Portfolio Management

## Helper Functions

In [40]:
# Merger and hold data for all dataframes with the same column name
def merge_df_by_column_name(column_name, sdate, edate, *tickers):
    
    df_result = pd.DataFrame()
    
    for x in tickers:
        df = create_df(x)
        
        mask = (df.index >= sdate) & (df.index <= edate)
        df_result[x] = df.loc[mask][column_name]
        
    return df_result

In [41]:
def calculate_cummulative_return_porfolio(df):
    tickers = []
    return_cum = []
    for _, r in df.iterrows():
        df_new = create_df(r['Symbol'])
        if df_new is None:
            pass
        else:
            tickers.append(r['Symbol'])
            cum = df_new['return_cummulative'].iloc[-1]
            return_cum.append(cum)

    return pd.DataFrame({'Ticker': tickers, 'return_cummulative': return_cum})


In [42]:
df_main = pd.read_csv('SP500.csv')

technology = df_main.loc[df_main['Sector'] == 'Information Technology']
industrial = df_main.loc[df_main['Sector'] == 'Industrials']
healthcare = df_main.loc[df_main['Sector'] == 'Health Care']
consumer = df_main.loc[df_main['Sector'] == 'Consumer Discretionary']
utilities = df_main.loc[df_main['Sector'] == 'Utilities']
financials = df_main.loc[df_main['Sector'] == 'Financials']
materials = df_main.loc[df_main['Sector'] == 'Materials']
realestate = df_main.loc[df_main['Sector'] == 'Real Estate']
staples = df_main.loc[df_main['Sector'] == 'Consumer Staples']
energy = df_main.loc[df_main['Sector'] == 'Energy']
communication = df_main.loc[df_main['Sector'] == 'Telecommunication Services']

technology = calculate_cummulative_return_porfolio(technology)
industrial = calculate_cummulative_return_porfolio(industrial)
healthcare = calculate_cummulative_return_porfolio(healthcare)
consumer = calculate_cummulative_return_porfolio(consumer)
utilities = calculate_cummulative_return_porfolio(utilities)
financials = calculate_cummulative_return_porfolio(financials)
materials = calculate_cummulative_return_porfolio(materials)
realestate = calculate_cummulative_return_porfolio(realestate)
staples = calculate_cummulative_return_porfolio(staples)
energy = calculate_cummulative_return_porfolio(energy)
communication = calculate_cummulative_return_porfolio(communication)

CSV for FLIR does not exist
CSV for HRS does not exist
CSV for RHT does not exist
CSV for SYMC does not exist
CSV for TSS does not exist
CSV for JEC does not exist
CSV for RTN does not exist
CSV for UTX does not exist
CSV for ALXN does not exist
CSV for AGN does not exist
CSV for CELG does not exist
CSV for MYL does not exist
CSV for VAR does not exist
CSV for CBS does not exist
CSV for LB does not exist
CSV for KORS does not exist
CSV for PCLN does not exist
CSV for TIF does not exist
CSV for VIAB does not exist
CSV for WYN does not exist
CSV for BBT does not exist
CSV for BRK.B does not exist
CSV for ETFC does not exist
CSV for LUK does not exist
CSV for STI does not exist
CSV for TMK does not exist
CSV for DWDP does not exist
CSV for CBG does not exist
CSV for GGP does not exist
CSV for HCN does not exist
CSV for BF.B does not exist
CSV for DPS does not exist
CSV for APC does not exist
CSV for BHGE does not exist
CSV for COG does not exist
CSV for XEC does not exist
CSV for CXO does

In [45]:
port_list = ['A', 'AAPL', 'DFS', 'FTV',
'AMD', 'GPS', 'GPC', 'MDT', 'MET',
'NKE', 'RCL']

mult_df = merge_df_by_column_name('Close',  SDATE, EDATE, *port_list)
mult_df

TypeError: '>=' not supported between instances of 'numpy.ndarray' and 'numpy.ndarray'